## Import libraries

In [114]:
import pandas as pd
import numpy as np
from typing import Optional, List, Tuple
import random
import time
import math
import synthetic_data_generators as sdg

import time

## Import Synthetic Data

In [115]:
synth_network = pd.read_csv("../data/synth_network.csv")
synth_pool = pd.read_csv("../data/synth_pool.csv")
synth_members = pd.read_csv("../data/synth_members.csv")
synth_reqs = sdg.synth_reqs

In [116]:
print(f"Network size: {len(synth_network)}")
print(f"Pool size: {len(synth_pool)}")
print(f"Number of members: {len(synth_members)}")
synth_reqs

Network size: 100
Pool size: 100
Number of members: 1000


,specialty,county,provider_count,distance_req,min_access_pct,min_providers
0,cardiologist,wayne,1,15,90,5
1,pcp,wayne,2,15,90,10
2,ent,wayne,1,15,90,5
3,urologist,wayne,1,15,90,5
4,obgyn,wayne,1,15,90,5


In [188]:
class NetworkOptimizer:
    """
Class that can perform provider network optimization. Implementation is a steepest-ascent, Hill-climbing local search 
optimization algorithm.

>>> optimizer = NetworkOptimizer(pool, members, adequacy_reqs)
>>> optimizer.optimize(num_rounds)
    """
    def __init__(self, 
                 pool: pd.DataFrame,
                 members: pd.DataFrame,
                 adequacy_reqs: pd.DataFrame,
                 user_objective: Optional[callable] = None,
                 network: Optional[pd.DataFrame] = None
                 ) -> None:
        """
    Initialize the optimizer. The pool, members, and adequacy requirements are required. If not passed, the network will start as blank and the optimizer
    will optimize it, guided by the objective function.
    
    :param pool: pandas DataFrame storing the pool of potential providers the network can have
    :param members: pandas DataFrame storing the members (beneficiaries) that the network serves
    :param adequacy_reqs: pandas DataFrame storing the adequacy requirements for the network
    :param objective: objective function that takes in a pandas dataframe and guides the algorithm
    :param network: pandas DataFrame storing the providers already contracted for the network, if any
        """
        
        self.initial_pool = pool.copy()
        self.adequacy_reqs = adequacy_reqs.copy()
        self.members = members.copy()
        self.user_objective = user_objective
        self.initial_network = network.copy() if network is not None else pd.DataFrame(columns=self.initial_pool.columns)
        self.distance_matrix = np.empty(0)
        self.performance_history = np.empty(0)
        self.move_tracker = []
        self.time_tracker = np.empty(0)
        self.total_optimization_rounds = 0
        self.adequacy_detail = np.empty(0)

        # create access listing
        if len(self.initial_network) > 0:
            self.access_listing = self.members.rename(
                {"latitude":"member_latitude", 
                "longitude":"member_longitude",
                "county":"member_county"}, axis=1).merge(pd.concat([self.initial_network, self.initial_pool]).rename(
                    {"latitude":"provider_latitude",
                    "longitude":"provider_longitude",
                    "county":"provider_county"}, axis=1), how="cross")

        else:
            self.access_listing = self.members.rename(
                {"latitude":"member_latitude", 
                "longitude":"member_longitude",
                "county":"member_county"}, axis=1).merge(self.initial_pool.rename(
                    {"latitude":"provider_latitude",
                    "longitude":"provider_longitude",
                    "county":"provider_county"}, axis=1), how="cross")

        def haversine(lat1, lon1, lat2, lon2):
     
            # distance between latitudes
            # and longitudes
            dLat = (lat2 - lat1) * math.pi / 180.0
            dLon = (lon2 - lon1) * math.pi / 180.0
        
            # convert to radians
            lat1 = (lat1) * math.pi / 180.0
            lat2 = (lat2) * math.pi / 180.0
        
            # apply formulae
            a = (pow(math.sin(dLat / 2), 2) +
                pow(math.sin(dLon / 2), 2) *
                    math.cos(lat1) * math.cos(lat2))
            rad = 3959
            c = 2 * math.asin(math.sqrt(a))
            return rad * c
        
        self.access_listing["distance"] = self.access_listing.apply(lambda row: haversine(row.member_latitude, 
                                                                                          row.member_longitude, 
                                                                                          row.provider_latitude, 
                                                                                          row.provider_longitude), axis=1).round(1)

        self.access_listing = self.access_listing.merge(self.adequacy_reqs, left_on=["member_county", "specialty"], right_on=["county", "specialty"], how="left")
        
        self.pct_serving = (self.access_listing[["location_id"]].merge(
            (self.access_listing.loc[self.access_listing.distance <= self.access_listing.distance_req].groupby("location_id")["member_id"].count() / len(self.members))
            .reset_index()
            .rename({"member_id":"pct_serving"}, axis=1), on="location_id", how="left").fillna(0).drop_duplicates().reset_index(drop=True))

        self.pool = pool.copy().merge(self.pct_serving, on=["location_id"], how="left")

        self.best_network = network.copy().merge(self.pct_serving, on=["location_id"], how="left") if network is not None else pd.DataFrame(columns=self.pool.columns)

    def adequacy(self, network: pd.DataFrame) -> float:
        """
    Calculate adequacy of a network using the adequacy requirements provided by the user. The returned value is a float that
    is a slight modification of the adequacy index score. It takes the network and the adequacy requirements and it returns the mean
    of the product of the percent of members with access and the percent of required providers for all the county/specialty combinations.

    :param network: pandas DataFrame with the network for which you want to calculate adequacy 
        """
        if len(network) == 0:
            return 0
        else:
            network_distances = (self.access_listing[["member_id", "npi", "location_id", "distance", "provider_count"]]
                     .merge(network[["npi", "location_id", "specialty", "county"]], on=["npi", "location_id"], how="inner")
                     .merge(self.adequacy_reqs[["county", "specialty", "distance_req"]], on=["county", "specialty"], how="left"))

            network_distances["meets_distance"] = network_distances.distance <= network_distances.distance_req

            access_detail = (network_distances
                         .loc[network_distances.meets_distance == True]
                         .groupby(["member_id", "county", "specialty", "provider_count"])[["npi"]]
                         .count()
                         .reset_index()
                         .rename({"npi":"servicing_provider_count"}, axis=1))
            
            member_access_summary = (access_detail
                         .loc[access_detail.servicing_provider_count >= access_detail.provider_count]
                         .groupby(["county", "specialty"])[["member_id"]]
                         .nunique()
                         .reset_index()
                         .rename({"member_id":"members_with_access"}, axis=1))
            
            servicing_provider_summary = (network_distances
                              .groupby(["county", "specialty"])[["npi"]]
                              .nunique()
                              .reset_index()
                              .rename({"npi":"servicing_providers"}, axis=1))
            
            adequacy_detail = (self.adequacy_reqs.merge(self.members
                                .groupby("county")[["member_id"]]
                                .count()
                                .reset_index()
                                .rename({"member_id":"total_members"}, axis=1), how="left", on="county")
                               .merge(member_access_summary, how="left", on=["county", "specialty"])
                               .merge(servicing_provider_summary, how="left", on=["county", "specialty"])
                               .fillna(0))
            
            adequacy_detail["pct_with_access"] = adequacy_detail.members_with_access / adequacy_detail.total_members

            adequacy_detail["adequacy_index"] = adequacy_detail.pct_with_access * (adequacy_detail.servicing_providers / adequacy_detail.min_providers).apply(lambda x: min(1, x))

            self.adequacy_detail = adequacy_detail
            
            return adequacy_detail.adequacy_index.mean()
        
    def objective(self, network: pd.DataFrame) -> float:
        """
    Objective function that describes the goal of the optimization. Takes in a pandas DataFrame storing a provider network as input.
    It is the compass for the algorithm to optimize the network. The default is adequacy, but if the user passes in a function, it will use that instead.

    :param network: pandas DataFrame with the network for which you want to calculate performance
        """
        return self.adequacy(network) if self.user_objective is None else self.user_objective(self, network)

    def successor(self, network: pd.DataFrame, pool: pd.DataFrame) -> List[Tuple]:
        """
    Returns all possible moves as successor states, given the provided network and provider pool. This represents
    all possible changes to the network. If empty network, all successor states are simply the states with each pool provider
    added to the network. Important to note that each successor state only deals with one change of a provider, i.e. the smallest
    possible "step" you can take.

    :param network: pandas DataFrame with the network in its current state
    :param pool: pandas DataFrame with the pool of potential providers
        """
        # get additions
        # additions = [("addition", idx) for idx in pool.sort_values(by="pct_serving", ascending=False).index.to_list()]
        # additions = [("addition", group) for group in pool.groupby("group_id")[["pct_serving"]].mean().reset_index().sort_values(by="pct_serving", ascending=False).group_id.to_list()]
        additions = [("addition", group) for group in pool.group_id.drop_duplicates().to_list()]
        return additions
        
        # if len(network) == 0:
        #     removals = [(None, None) for i in range(len(additions))]
        #     swaps = [(None, None, None) for i in range(len(additions))]
        #     return list(zip(additions, removals, swaps))
        # else:
        #     # get replacement combinations within specialty
        #     swapDF = network.reset_index().merge(pool.reset_index(), on = ["county", "specialty"], how = "inner", suffixes= ["_network", "_pool"]).dropna()
        #     swapDF["pct_diff"] = swapDF.pct_serving_pool - swapDF.pct_serving_network
        #     swaps = [("swap",i,j) for i,j in swapDF.sort_values(by="pct_diff", ascending=False)[["index_network", "index_pool"]].values]
        #     # get removals
        #     removals =  [("removal", idx) for idx in network.sort_values(by="pct_serving", ascending=True).index.astype(int).to_list()]
        #     return list(zip(additions, removals, swaps))
    
    def create_state(self, network: pd.DataFrame, pool: pd.DataFrame, change: Tuple):
        """
    Create a network based on the pool and the change. This function takes the network passed in and creates a copy, then 
    makes the change that is described by the change parameter, which comes as a tuple. The first element of the tuple
    tells you what kind of change: addition, removal, or a swap. The rest of the tuple tells you the index of the row that is associated 
    with the change.

    :param network: pandas DataFrame with the network in its current state
    :param pool: pandas DataFrame with the pool of potential providers
    :param change: tuple that describes the change to be made to the network
        """
        # make a true copy of the network, not just a reference
        # see here: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html
        new_network = network.copy()

        if change[0] == "addition":
            new_network = pd.concat([new_network, pool.loc[pool.group_id == change[1]]]).reset_index(drop=True)
            # new_network.loc[len(new_network)] = pool.loc[change[1]]
            return new_network
        # elif change[0] == "removal":
        #     new_network = new_network.drop(change[1])
        #     return new_network
        # else:
        #     new_network.loc[change[1]] = pool.loc[change[2]]
        #     return new_network
    
    def optimize(self, num_rounds: int) -> None:
        """
    Perform steepest-ascent local search optimization for a number of rounds input by the user.
    The algorithm takes the network, determines all the possible moves from the successor function. Then
    it calculates the performance of all the successor states and stores the best one. Repeat for number of
    rounds or until goal state is met.

    :param num_rounds: max number of rounds to perform optimization
        """
        # save performance of initial network
        if len(self.performance_history) == 0:
            self.performance_history = np.append(self.performance_history, self.objective(self.initial_network))

        # for set number of rounds
        for optim_round in range(num_rounds):
            self.total_optimization_rounds += 1
            start = time.perf_counter()
            print(f"Optimization round {self.total_optimization_rounds} ...")
            # get state changes
            state_changes = self.successor(self.best_network, self.pool)
            if len(state_changes) == 0:
                print("Pool has been exhaused")
                break

            new_states = [self.create_state(self.best_network, self.pool, change) for change in state_changes]
            new_scores = [self.objective(state) for state in new_states]
            best_score = np.max(new_scores)
            best_state_idx = np.argmax(new_scores)
            best_move = state_changes[best_state_idx]
            best_state = new_states[best_state_idx]
            
            if best_score > self.performance_history[-1]:
                self.performance_history = np.append(self.performance_history, best_score)
                self.move_tracker.append(best_move)
                self.best_network = best_state
                if best_move[0] == "addition":
                        change_members = self.pool.loc[self.pool.group_id == best_move[1]].index
                        self.pool.drop(change_members, inplace = True)
            
            # for change in state_changes:
                # generate new states for the options and get the scores
                # new_state = self.create_state(self.best_network, self.pool, change)
                # new_score = self.objective(new_state)

                # new_states = [self.create_state(self.best_network, self.pool, option) for option in options if option[0] in ["addition", "removal", "swap"]]
                # new_scores = np.array([self.objective(state) for state in new_states])

                # get the best performing state and score
                # best_score = np.max(new_scores)
                # best_state_idx = np.argmax(new_scores)

                # best_move = options[best_state_idx]
                # best_state = new_states[best_state_idx]

                #if the best new move is better than the latest best performance
                # if new_score > self.performance_history[-1]:
                    # store best performance and move
                    # self.performance_history = np.append(self.performance_history, new_score)
                    # self.move_tracker.append(change)
                    # update the best network
                    # self.best_network = new_state
                    # if move involves a pool provider
                    # drop the new provider from pool so that the algo knows not to try that one again
                    # if best_move[0] == "swap":
                    #     self.pool.drop(best_move[2], inplace = True)
                    # if change[0] == "addition":
                    #     self.pool.drop(change[1], inplace = True)
                    # stop = time.perf_counter()
                    # self.time_tracker = np.append(self.time_tracker, stop - start)
                    # break
                
            if len(self.move_tracker) < self.total_optimization_rounds:
                stop = time.perf_counter()
                self.time_tracker = np.append(self.time_tracker, stop - start)
                print("No more options for optimization")
                break

            stop = time.perf_counter()
            self.time_tracker = np.append(self.time_tracker, stop - start)

        print(f"Average seconds per round of optimization: {self.time_tracker.mean().round(1)}")
        print(f"Adequacy score for best network: {self.adequacy(self.best_network)}\n")

In [189]:
optimizer = NetworkOptimizer(pool=synth_pool, 
                             members=synth_members, 
                             adequacy_reqs=synth_reqs, 
                            #  network=synth_network
                             )

In [198]:
optimizer.optimize(15)

Optimization round 11 ...
No more options for optimization
Average seconds per round of optimization: 0.8
Adequacy score for best network: 0.9736



In [199]:
optimizer.move_tracker

[('addition', 4),
 ('addition', 2),
 ('addition', 14),
 ('addition', 0),
 ('addition', 10),
 ('addition', 13),
 ('addition', 5),
 ('addition', 9),
 ('addition', 11)]

In [200]:
optimizer.performance_history

array([0.     , 0.25128, 0.4768 , 0.65592, 0.74724, 0.80772, 0.87188,
       0.94692, 0.9714 , 0.9736 ])

In [201]:
optimizer.adequacy_detail

,specialty,county,provider_count,distance_req,min_access_pct,min_providers,total_members,members_with_access,servicing_providers,pct_with_access,adequacy_index
0,cardiologist,wayne,1,15,90,5,1000,1000,11,1.000,1.000
1,pcp,wayne,2,15,90,10,1000,990,10,0.990,0.990
2,ent,wayne,1,15,90,5,1000,1000,8,1.000,1.000
3,urologist,wayne,1,15,90,5,1000,910,7,0.910,0.910
4,obgyn,wayne,1,15,90,5,1000,968,6,0.968,0.968


In [202]:
optimizer.best_network

,npi,specialty,group_id,efficiency,effectiveness,location_id,county,latitude,longitude,pct_serving
0,72,cardiologist,4,5,1,6,wayne,42.000698,-83.751425,0.334
1,56,urologist,4,1,1,7,wayne,42.199648,-83.761273,0.628
2,89,cardiologist,4,1,1,28,wayne,42.153219,-83.988858,0.323
3,27,ent,4,1,2,45,wayne,42.286955,-83.598814,0.587
4,81,cardiologist,4,4,1,46,wayne,42.296682,-83.854589,0.548
...,...,...,...,...,...,...,...,...,...,...
59,48,pcp,9,3,2,77,wayne,42.373220,-83.823846,0.480
60,93,cardiologist,9,1,3,92,wayne,42.223080,-83.734750,0.649
61,59,ent,11,4,4,42,wayne,42.354035,-83.533698,0.449
62,3,obgyn,11,4,4,58,wayne,41.936133,-83.595134,0.196


In [204]:
len(optimizer.pool)

36